<a href="https://colab.research.google.com/github/jtwya/AOP/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義


*   將 性別 欄位 Label Encoding

*   將客戶分成 3 群，並解讀各群特質

*   計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值









## 資料收集

In [ ]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv

In [1]:
import pandas as pd
url ='https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd.read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


## 資料前處理

### 資料清理

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


### 探索性分析

In [3]:
df_cor = df.drop(columns=['性別']).corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


### 資料分割

### 類別轉換

In [4]:
df['性別'].unique()

array(['女', '男'], dtype=object)

In [5]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['女','男']])
df[['性別']] = encoder.fit_transform(df[['性別']])

### 特徵縮放

## 模型訓練

In [12]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

KMeans(n_clusters=3)

In [13]:
# 為每筆資料加上分群結果
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,1
1,0.0,51,71,91,1
2,0.0,30,65,10,2
3,0.0,88,49,17,2
4,0.0,55,48,70,1
...,...,...,...,...,...
195,1.0,86,84,82,1
196,1.0,59,52,30,2
197,0.0,63,29,61,1
198,1.0,67,80,9,2


## 模型評估

In [14]:
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.584906,28.962264,55.226415,72.018868
1,0.552239,67.641791,58.985075,75.104478
2,0.575000,56.187500,61.075000,20.787500


In [15]:
# 評估分群效果
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns = 'cluster'), kmeans.labels_)
score

np.float64(89.34113816481202)

## 模型調整

## 模型部署

In [16]:
df_nocluster = df.drop(columns='cluster')
for i in range(2,16):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(df_nocluster)
  score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
  print(f'k={i}, score={score}')

k=2, score=111.68583137120146
k=3, score=96.52156233412978
k=4, score=90.83142449293072
k=5, score=81.86505162710102
k=6, score=89.14804932309094
k=7, score=91.13485889352098
k=8, score=98.55120019984643
k=9, score=95.21613708648951
k=10, score=93.87402415370205
k=11, score=90.72455243873873
k=12, score=89.72696645035789
k=13, score=87.81120037838417
k=14, score=82.18324074590896
k=15, score=81.68074702573358


### 儲存模型

### 推論預測